# Chapter 6. Odds and Addends
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap06.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import DataFrames as pd # pandas in Python
import Distributions as dst
import Plots as plts

In [ ]:
include("./pmf.jl")
import .ProbabilityMassFunction as pmf

### Odds

In [ ]:
function prob2odds(prob::T)::T where T<:Union{Float64, Rational}
    @assert (0 <= prob <= 1)
    return prob / (1-prob)
end

In [ ]:
prob2odds(0.75)

In [ ]:
prob2odds(1//10) |> Float64

In [ ]:
prob2odds(9//10) |> Float64

In [ ]:
function odds2prob(odds::T)::T where T<:Union{Float64, Rational}
   @assert 0 <= odds
   return odds / (odds + 1) 
end


For practice I will try to derive the formula (see odds2prob).
Let's go.

$odds = \frac{p}{1-p}$

$\frac{p}{1-p} = odds$

$p = odds * (1-p)$

$p = odds - odds*p$

$p + odds*p = odds$

$p * (1+odds) = odds$

$p = \frac{odds}{1+odds}$

$p = \frac{odds}{odds+1}$

In [ ]:
odds2prob(3/2)

In [ ]:
function odds2prob(yes::T, no::T)::T where T<:Union{Float64, Rational}
    @assert yes >= 0
    @assert no > 0
    return yes / (yes+no)
end

In [ ]:
function odds2prob(yes::Int, no::Int)::Float64
    @assert yes >= 0
    @assert no > 0
    return yes / (yes+no)
end

In [ ]:
odds2prob(3, 2)

In [ ]:
odds2prob(3//5, 2//5) |> Float64

### Bayes's Rule

Bayes's theorem in the "probability form":

$P(H|D) = \frac{P(H) P(D|H)}{P(D)}$

Writing odds(A) for odds in favor of A, we can express Bayes's Theorem in "odds form":

$odds(A|D) = odds(A) \frac{P(D|A)}{P(D|B)}$

The posterior odds are the prior odds times the likelihood ratio.

**My comment**

OK, although I'm not sure how we derived that and why we resigned from H and got A and B, all of a sudden.

Let's get back to the cookie problem. From Chapter 2 (The Bayes's Theorem).

We got two bowls:
- Bowl 1 contains 30 vanilla cookies and 10 chocolate cookies
- Bowl 2 contains 20 vanilla cookies and 20 chocolate cookies

You choose one bowl at random and choose the cookie at random. If the cookie is vanilla, what is the probability that it came from Bowl 1?

The prior probability is 0.5, so the prior odds are 1 [odds(A)]

The likelihood ratio is 3/4 [P(D|A)] and 2/4 [P(D|B)]

The posterior odds are:

$odds(A|D) = 1 * \frac{3/4}{2/4}$

Which is equal to:


In [ ]:
1 * (3//4) / (2//4)

And now for the posterior probability

In [ ]:
odds2prob(3, 2)

If we draw another cookie and it's chocolate, we can do another update:

In [ ]:
3//2 * 1//4 / 1//2

In [ ]:
odds2prob(3, 4)

### Oliver's Blood

From [MacKay’s Information Theory, Inference, and Learning Algorithms](https://www.inference.org.uk/mackay/itila/):

Two people have left traces of their own blood at the scene of a crime. A suspect, Oliver, is tested and found to have type ‘O’ blood. The blood groups of the two traces are found to be of type ‘O’ (a common type in the local population, having frequency 60%) and of type ‘AB’ (a rare type, with frequency 1%). Do these data [the traces found at the scene] give evidence in favor of the proposition that Oliver was one of the people [who left blood at the scene]?


To make our reasoning easier we can transform Bayes's Rule:

$odds(A|D) = odds(A) \frac{P(D|A)}{P(D|B)}$

Dividing through odds(A), we get:

$\frac{odds(A|D)}{odds(A)} = \frac{P(D|A)}{P(D|B)}$

The term on the left is the ratio of the posterior and prior odds.

The term on the right is the likelihood ratio, also called the **Bayes factor**

If the Bayes factor is greater than 1, that means that the data were more likely under A than under B. And that means that the odds are greater, in light of the data, than they were before.

If the Bayes factor is less than 1, that means the data were less likely under A than under B, so the odds in favor of A go down.

Finally, if the Bayes factor is exactly 1, the data are equally likely under either hypothesis, so the odds do not change.

In [ ]:
# if Oliver is guilty we need to account for the AB blood type
like1 = 0.01 # P(D|A)
# if Oliver isn't guilty we need to account for both samples
# so P(blood O) and P(blood AB) or P(blood AB) and P(blood O)
like2 = 2 * 0.6 * 0.01 # P(D|B)

likelihood_ratio = like1 / like2

In [ ]:
# if the prior odds [odds(A)] were 1
# then posterior odds [odds(A|B)] would be
odds2prob(likelihood_ratio)

#### Oliver's Blood Exercise

Suppose that based on other evidence, your prior belief in Oliver's guilt is 90%. How much would the blood evidence in this section change your beliefs? What if you initially thought there was only a 10% chance of this guilt?

So, our formula is:

$odds(A|D) = odds(A) \frac{P(D|A)}{P(D|B)}$

So, previously prior odds [odds(A) were 1, we assumed it equally likely that Oliver accounted for the blood sample O (p = 0.5) as that he did not (p = 0.5)].

And now the probablity is 0.9 (90%), so the prior odds [odds(A)] are
0.9/0.1 = 9.

Or the probability is 0.1 (10%), so the prior odds [odds(A)] are 0.1/0.9 = 0.11(1).

Given the above, the posterior odds [odds(A|D)] are:

$\frac{0.9}{0.1} * \frac{P(D|A)}{P(D|B)}$

or

$\frac{0.1}{0.9} * \frac{P(D|A)}{P(D|B)}$

And the probabilities are:

In [ ]:
odds2prob(0.9/0.1 * like1/like2)

In [ ]:
odds2prob(0.1/0.9 * like1/like2)

### Addends

This part of the chapter is about distributions of sums and results of other operations.

For now we will deal with the problem where we are given the inputs and compute the distribution of the output.

As a first example, suppose you roll two dice and add them up. What is the distribution of the sum?

In [ ]:
function mk_die(no_of_sides::Int)::pmf.Pmf{Int}
    @assert no_of_sides > 0
    return pmf.mk_pmf_from_seq(collect(1:no_of_sides))
end

In [ ]:
die = mk_die(6)

In [ ]:
plts.bar(die.names, die.priors, legend=false)
plts.title!("Throwing one six sided die")
plts.xlabel!("Outcome")
plts.ylabel!("PMF")

In [ ]:
function add_dist(pmf1::pmf.Pmf{Int}, pmf2::pmf.Pmf{Int})::pmf.Pmf{Int}
    res::Dict{Int, Float64} = Dict()
    n::Int = 0
    p::Float64 = 0.0
    for i in eachindex(pmf1.names)
        for j in eachindex(pmf2.names)
            n = pmf1.names[i] + pmf2.names[j]
            p = pmf1.priors[i] * pmf2.priors[j]
            res[n] = get(res, n, 0) + p
        end
    end
    ordered_keys::Vector{Int64} = sort(collect(keys(res)))
    ordered_vals::Vector{Float64} = [res[k] for k in ordered_keys]
    return pmf.Pmf(ordered_keys, ordered_vals ./ sum(ordered_vals))
end

In [ ]:
twice = add_dist(die, die);

In [ ]:
plts.bar(twice.names, twice.priors, legend=false)
plts.title!("Throwing two six sided dice")
plts.xlabel!("Outcome")
plts.ylabel!("PMF")
plts.xlims!((0, 13))
plts.xticks!(2:12)

And how about 3 dice?

In [ ]:
function add_dist_seq(seq::Vector{<:pmf.Pmf{Int}})::pmf.Pmf{Int}
    res::pmf.Pmf{Int} = seq[1]
    for i in 2:length(seq)
        res = add_dist(res, seq[i])
    end
    return res
end 

In [ ]:
three_dice = repeat([die], 3);

In [ ]:
thrice = add_dist_seq(three_dice);

In [ ]:
plts.bar(thrice.names, thrice.priors, legend=false)
plts.title!("Throwing three six sided dice")
plts.xlabel!("Outcome")
plts.ylabel!("PMF")
plts.xlims!((2, 19))
plts.xticks!(3:18)

In [ ]:
pmf.draw_priors(die, "Distributions of sums", "Outcome", "PMF", "once")
plts.plot!(twice.names, twice.priors, label="twice")
plts.plot!(thrice.names, thrice.priors, label="thrice")

[...] this example demonstrates the Central Limit Theorem, which says that the distribution of a sum converges on a bell-shaped normal distribution, at least under some conditions.

### Gluten Sensitivity

In a [scientific paper](https://onlinelibrary.wiley.com/doi/full/10.1111/apt.13372) the authors tested people with gluten sensitivity if they can differentiate flour with and without gluten (blind test).

Out of 35 subjects, 12 correctly identified the gluten flour based on resumption of symptoms while they were eating it. Another 17 wrongly identified the gluten-free flour based on their symptoms, and 6 were unable to distinguish.

So here’s the question: based on this data, how many of the subjects are sensitive to gluten and how many are guessing?

[...] first we have to make some modeling decisions. I’ll (Allen B. Downey) assume:
- People who are sensitive to gluten have a 95% chance of correctly identifying gluten flour under the challenge conditions, and
- People who are not sensitive have a 40% chance of identifying the gluten flour by chance (and a 60% chance of either choosing the other flour or failing to distinguish). 

I (Allen B. Downey) will solve this problem in two steps. First, assuming that we know how many subjects are sensitive, I (Allen B. Downey) will compute the distribution of the data. Then, using the likelihood of the data, I (Allen B. Downey) will compute the posterior distribution of the number of sensitive patients.

The first is the **forward problem**; the second is the **inverse problem**.

### The Forward Problem

Suppose we know that 10 of the 35 subjects are sensistive to gluten. That means that 25 are not:

In [ ]:
n = 35
num_sensitive = 10
num_insensitive = n - num_sensitive

Each sensitive subject has a 95% chance of identifying the gluten fluor, so the number of correct identifications follows a binomial distribution.

In [ ]:
dist_sensitive = pmf.mk_binomial_pmf(num_sensitive, 0.95);
dist_insensitive = pmf.mk_binomial_pmf(num_insensitive, 0.4);

Now we can use `add_dist` to compute the distribution of the total number of correct identifications:

In [ ]:
dist_total = add_dist(dist_sensitive, dist_insensitive);

In [ ]:
pmf.draw_priors(dist_sensitive, "Gluten sensitivity",
    "Number of correct identifications", "PMF", "sensitive")
plts.plot!(dist_insensitive.names, dist_insensitive.priors,
    label="insensitive")
plts.plot!(dist_total.names, dist_total.priors, label="total")

We expect most of the sensitive subjects to identify the gluten flour correctly. Of the 25 insensitive subjects, we expect about 10 to identify the gluten flour by chance. So we expect about 20 correct identifications in total.

This is the answer to the forward problem: given the number of sensitive subjects, we can compute the distribution of the data.

### The Inverse Problem

Now given the data, we'll compute the posterior distribution of the number of sensistive subjects.

In [ ]:
dict_num_sens_pmf = Dict{Int, pmf.Pmf{Int}}()
# previously we defined n to be 35
# (number of participants in the experiment)
for num_sens in 0:n
    num_insens = n - num_sens
    dist_sens = pmf.mk_binomial_pmf(num_sens, 0.95)
    dist_insens = pmf.mk_binomial_pmf(num_insens, 0.4)
    dist_total = add_dist(dist_sens, dist_insens)
    dict_num_sens_pmf[num_sens] = dist_total
end

In [ ]:
p = pmf.draw_priors(dict_num_sens_pmf[0], "Gluten sensitivity",
    "Number of correct identifications", "PMF", "num_sensitive = 0")

for n_sens in 10:10:30
    plts.plot!(p, dict_num_sens_pmf[n_sens].names,
    dict_num_sens_pmf[n_sens].priors, label="num_sensitive = $n_sens")
end

p

In the experiment the authors reported 12 correct identifications of gluten flour.

Let's compare it with other possible identification number (e.g. 20).

In [ ]:
# names in pmf are 0:35, so ind 13 is for 12 correct identifictions
correct12 = [dict_num_sens_pmf[i].priors[13] for i in 0:35]
# n is 35, number of participants in the experiment
gluten12 = pmf.mk_pmf_from_seq(collect(0:n))
pmf.update_likelihoods!(gluten12, correct12)
pmf.update_posteriors!(gluten12);

In [ ]:
# names in pmf are 0:35, so ind 21 is for 20 correct identifictions
correct20 = [dict_num_sens_pmf[i].priors[21] for i in 0:35]
# n is 35, number of participants in the experiment
gluten20 = pmf.mk_pmf_from_seq(collect(0:n))
pmf.update_likelihoods!(gluten20, correct20)
pmf.update_posteriors!(gluten20);

In [ ]:
pmf.draw_posteriors(gluten12, "Posterior distributionn",
    "Number of sensitive subjects", "PMF",
    "posterior with 12 correct")
plts.plot!(gluten20.names, gluten20.posteriors,
    label="posterior with 20 correct")

In [ ]:
pmf.get_name_max_posterior(gluten12)

In [ ]:
pmf.get_name_max_posterior(gluten20)

## Exercises

### Exercise 1

Let's use Bayes's Rule to solve the Elvis problem from [Ch3. Distributions](https://allendowney.github.io/ThinkBayes2/chap03.html#exercises):

Elvis Presley had a twin brother (who died at birth). What is the probability that Elvis was an identical twin?

Hint: In 1935, about 2/3 of twins were fraternal and 1/3 were identical.

#### Ex1. Reasoning

So, the Bayes's Rule is:

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$,

where:
- odds(A|D) are odds(identical twins | 2 brothers)
- odds(A) are odds(identical twins)
- P(D|A) is P(2 brothers | identical twins)
- P(D|B) is P(2 brothers | fraternal twins)

If, so then:
- odds(A) is 1/2 ($\frac{1/3}{2/3}$)
- P(D|A) is 1/2
- P(D|B) is 1/4

### Ex1. Solution

In [ ]:
ex1_odds = 0.5 * (0.5 / 0.25)

In [ ]:
odds2prob(ex1_odds)

### Ex1. Post-solution notes

Not sure why the author (Allen Downey) presents in the solution to this exercise the probability to be equal 0.5(5)...., especially that in [Ch3. Distributions](https://allendowney.github.io/ThinkBayes2/chap03.html#exercises) the probability is 0.5, so 0.5(5)... presented here is not quite compatible

### Exercise 2

The following is an [interview question that appeared on glassdoor.com](https://www.glassdoor.com/Interview/You-re-about-to-get-on-a-plane-to-Seattle-You-want-to-know-if-you-should-bring-an-umbrella-You-call-3-random-friends-of-y-QTN_519262.htm), attributed to Facebook:

You’re about to get on a plane to Seattle. You want to know if you should bring an umbrella. You call 3 random friends of yours who live there and ask each independently if it’s raining. Each of your friends has a 2/3 chance of telling you the truth and a 1/3 chance of messing with you by lying. All 3 friends tell you that “Yes” it is raining. What is the probability that it’s actually raining in Seattle?

Use Bayes’s Rule to solve this problem. As a prior you can assume that it rains in Seattle about 10% of the time.

#### Ex2. Reasoning

The Bayes's Rule is:

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$,

where:
- odds(A|D) are odds(rain | yyy)
- odds(A) are odds(rain)
- P(D|A) is P(yyy|rain)
- P(D|B) is P(yyy|no rain)

If, so then:
- odds(A) is prob2odds(1//10), so 1//9
- P(D|A) is (2//3)^3
- P(D|B) is (1//3)^3

#### Ex2. Solution

In [ ]:
ex2_odds = prob2odds(1//10) * ((2//3)^3) / ((1//3)^3)

In [ ]:
odds2prob(ex2_odds) |> Float64

### Exercise 3

[According to the CDC](https://www.cdc.gov/tobacco/data_statistics/fact_sheets/health_effects/effects_cig_smoking/), people who smoke are about 25 times more likely to develop lung cancer than nonsmokers.

[Also according to the CDC](https://www.cdc.gov/tobacco/data_statistics/fact_sheets/adult_data/cig_smoking/index.htm), about 14% of adults in the U.S. are smokers. If you learn that someone has lung cancer, what is the probability they are a smoker?

#### Ex3. Reasoning

The Bayes's Rule is:

$odds(A|D) = odds(A) * \frac{P(D|A)}{P(D|B)}$,

where:
- odds(A|D) are odds(smoker | lung cancer)
- odds(A) are odds(smoker)
- P(D|A) is P(lung cancer | smoker)
- P(D|B) is P(lung cancer | not smoker)

If, so then:
- odds(A) is prob2odds(14//100), so 7//43
- $\frac{P(D|A)}{P(D|B)}$ is 25


#### Ex3. Solution

In [ ]:
ex3_odds = (prob2odds(14//100) * 25)

In [ ]:
odds2prob(ex3_odds) |> Float64

### Exercise 3

In Dungeons & Dragons, the amount of damage a goblin can withstand is the sum of two six-sided dice. The amount of damage you inflict with a short sword is determined by rolling one six-sided die. A goblin is defeated if the total damage you inflict is greater than or equal to the amount it can withstand.

Suppose you are fighting a goblin and you have already inflicted 3 points of damage. What is your probability of defeating the goblin with your next successful attack?

Hint: You can use `Pmf.sub_dist` to subtract a constant amount, like 3, from a `Pmf`.

#### Ex3. Rationale and Solution

Hmm, what is `Pmf.sub_dist` and how it works.
Based on the code [here](https://github.com/AllenDowney/empiricaldist/blob/master/empiricaldist/empiricaldist.py) it seems that definition of my `sub_dist` should be:

<pre>
function sub_dist(dist::pmf.Pmf{Int}, x::Int)::pmf.Pmf{Int}
    return pmf.Pmf(dist.names .- x, dist.priors)
end
</pre>

So, for now let's go with that.

As for the vector version of `sub_dist` it seems it is the same as `add_dist` defined in this file, but `+` is replaced with `-` in the body of the function.

In [ ]:
function sub_dist(dist::pmf.Pmf{Int}, x::Int)::pmf.Pmf{Int}
    return pmf.Pmf(dist.names .- x, dist.priors)
end

In [ ]:
function sub_dist(pmf1::pmf.Pmf{Int}, pmf2::pmf.Pmf{Int})::pmf.Pmf{Int}
    res::Dict{Int, Float64} = Dict()
    n::Int = 0
    p::Float64 = 0.0
    for i in eachindex(pmf1.names)
        for j in eachindex(pmf2.names)
            n = pmf1.names[i] - pmf2.names[j]
            p = pmf1.priors[i] * pmf2.priors[j]
            res[n] = get(res, n, 0) - p
        end
    end
    ordered_keys::Vector{Int64} = sort(collect(keys(res)))
    ordered_vals::Vector{Float64} = [res[k] for k in ordered_keys]
    return pmf.Pmf(ordered_keys, ordered_vals ./ sum(ordered_vals))
end

In [ ]:
one_dice = mk_die(6) # damage
two_dice = add_dist(one_dice, one_dice) # amount it can withstand

In [ ]:
goblin = sub_dist(two_dice, 3) # we already inflicted 3 pts. of damage

In [ ]:
# we remove the values impossible to get after a throw of six sided dice (new damage)
# -1 and 0 (2 first names) are not possible for six sided dice
goblin = pmf.Pmf(goblin.names[3:end], goblin.priors[3:end])
# we normalize the priors so they add up to 1
goblin.priors = goblin.priors ./ sum(goblin.priors);

In [ ]:
# now we throw a dice to inflict damage on goblin
goblin = sub_dist(goblin, one_dice)

In [ ]:
# all names <= 0 are where goblin is dead after an attack
gob_dead_inds = findall(x -> x <= 0, goblin.names)
# we sum their probabilities to get the P(dead goblin)
goblin.priors[gob_dead_inds] |> sum

In [ ]:
gob_alive_inds = setdiff(eachindex(goblin.names), gob_dead_inds);

In [ ]:
plts.bar(goblin.names[gob_alive_inds], goblin.priors[gob_alive_inds], label="alive")
plts.bar!(goblin.names[gob_dead_inds], goblin.priors[gob_dead_inds], label="dead")
plts.title!("Probability of killing a goblin")
plts.xlabel!("HP remained")
plts.ylabel!("PMF")
plts.xticks!(collect(-5:1:8))

### Exercise 4

Suppose I have a box with a 6-sided die, an 8-sided die, and a 12-sided die. I choose one of the dice at random, roll it twice, multiply the outcomes, and report that the product is 12. What is the probability that I chose the 8-sided die?

Hint: `Pmf` provides a function called `mul_dist` that takes two `Pmf` objects and returns a `Pmf` that represents the distribution of the product.